# Importing the utils

In [ ]:
!pip install --upgrade diffusers accelerate transformers ipywidgets

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 48.1 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.50.3
    Uninstalling transformers-4.50.3:
      Successfully uninstalled transformers-4.50.3
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [ ]:
from IPython.display import display
from diffusers import DiffusionPipeline, DPMSolverMultistepScheduler
import torch
import numpy as np
import ipywidgets as widgets

In [ ]:
pipe_id = "stabilityai/stable-diffusion-xl-base-1.0"
pipe = DiffusionPipeline.from_pretrained(pipe_id, torch_dtype=torch.float16).to("cuda")
# More efficient Scheduler (need just 20 inference steps)
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
# trigger word : voxel style
pipe.load_lora_weights("Fictiverse/Voxel_XL_Lora", weight_name="VoxelXL_v1.safetensors", adapter_name="voxel")
# trigger word : toy_face
pipe.load_lora_weights("CiroN2022/toy-face", weight_name="toy_face_sdxl.safetensors", adapter_name="toy")

ValueError: PEFT backend is required for this method.

# Empty the Torch Cache

In [ ]:
torch.cuda.empty_cache()

# Creating the inteface

## Creating the widgets

In [ ]:
# Create widgets
prompt_widget = widgets.Text(
    value='A futuristic cityscape',
    placeholder='Enter your prompt here',
    description='Prompt:',
    disabled=False
)

seed_widget = widgets.IntText(
    value=0,
    min=-1,
    step=1,
    description='Seed (-1 is random):',
    disabled=False
)

inference_steps_widget = widgets.IntText(
    value=20,
    min=1,
    step=1,
    description='Inference Steps (for this Scheduler 20 is recommanded):',
    disabled=False
)

generate_button = widgets.Button(
    description='Generate Image',
    disabled=False,
    button_style='',  # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to generate image'
)

output_image = widgets.Output()

## Utils Functions

In [ ]:
def generate_image(b):
  print("in the function")
  with output_image:
    seed = seed_widget.value
    if seed == -1:
      seed = np.random.randint(0, 2**32 - 1)
    generator = torch.Generator(device="cuda").manual_seed(seed)
    output_image.clear_output()
    prompt = prompt_widget.value
    print("Generating image")
    image = pipe(prompt,
                  num_inference_steps=inference_steps_widget.value,
                  cross_attention_kwargs={"scale": 0.0},
                  generator=generator
                  ).images[0]
    print("Displaying image")
    display(image)

# Link the button to the function
generate_button.on_click(generate_image)

## Displaying the widgets

In [ ]:
# Display the widgets
display(prompt_widget)

Text(value='A futuristic cityscape', description='Prompt:', placeholder='Enter your prompt here')